Cuda 사용 가능한지 확인

In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

Data (CIFAR-10) 불러오기

In [3]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


Model 불러오기

In [8]:
from ADNNs.load_model import select_model

model = select_model("BranchyNet")
#model = select_model("SDNet")
#model = select_model("RANet")
#model = select_model("MSDNet")

#dynamic
#model = select_model("BlockDrop")
#model = select_model("SkipNet")


model.train()
model = model.cuda()

In [9]:
import torch.nn as nn
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Train

In [ ]:
for epoch in range(5): 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs[0].cuda(), labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:  # 매 200개 배치마다 출력
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 2.028
[1,   400] loss: 1.745
[1,   600] loss: 1.620
[1,   800] loss: 1.533


Test

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        
        outputs = model(images)
        _, predicted = torch.max(outputs[0].data, 1)
        total += labels.size(0)
        correct += (predicted.cuda() == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))